## Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from IPython.display import display, HTML
import sys
sys.path.append("../src")
from vibe_engine_full import VibeEngine

## Data

In [3]:
csv_path = "../data/spotify_tracks.csv"

df = pd.read_csv(csv_path)
df_tracks = df.dropna().reset_index(drop=True)
del df_tracks[df_tracks.columns[0]]
df_tracks.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


## NOTES
- pre-set sliders when user opens (randomized, need creative, fun, engaging call for action) or will this be the same as mode 2b?
- drop null, first col in df for all modes

## Initialization

In [4]:
engine = VibeEngine(df_tracks)

print("VibeEngine initialized.")
print("Num tracks:", engine.X.shape[0])
print("Num vibe dims:", engine.X.shape[1])
print("Vibe cols:", engine.vibe_cols)
print("Has track_id mapping:", len(engine.trackid_to_idx) > 0)

VibeEngine initialized.
Num tracks: 113999
Num vibe dims: 7
Vibe cols: ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 'instrumentalness', 'speechiness']
Has track_id mapping: True


## Mode 1

In [ ]:
from mode1_sliders_full import SliderVibeRecommender

mode1 = SliderVibeRecommender(df_tracks)

sliders = {
    "danceability": 80,
    "energy": 75,
    "valence": 70,
    "tempo": 65,
    "acousticness": 20,
    "instrumentalness": 10,
    "speechiness": 15,
}

recs_mode1 = mode1.recommend_by_sliders(
    sliders,
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

recs_mode1[["track_name", "artists", "track_genre", "vibe_score", "vibe_similarity"]].head(10)

,track_name,artists,track_genre,vibe_score,vibe_similarity
89912,Un Coco,Bad Bunny,reggaeton,0.939900,0.957376
89771,Sobrio,Maluma,reggaeton,0.934318,0.965397
3255,Feel Good Inc.,Gorillaz,alternative,0.929595,0.951994
89819,Se Preparó,Ozuna,reggaeton,0.919002,0.961253
81173,I Ain't Worried,OneRepublic,pop,0.907499,0.894374
20407,Right Now (Na Na Na),Akon,dance,0.899292,0.946615
51122,Satisfya,Imran Khan,hip-hop,0.898673,0.948341
104802,No Me Basta,India Martinez;Dvicio,spanish,0.896389,0.975486
39408,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.888171,0.900214
65109,Black Swan,BTS,k-pop,0.881009,0.916262


In [6]:
def show_vibe_recs(model, sliders, top_k=10, **kwargs):
    # 1. Get recommendations
    recs = model.recommend_by_sliders(
        sliders,
        top_k=top_k,
        **kwargs,
    )

    if recs.empty:
        print("No recommendations found for this vibe 😢 Try relaxing your filters.")
        return recs
    
def describe_vibe(sliders):
    d = sliders["danceability"]
    e = sliders["energy"]
    v = sliders["valence"]

    mood_bits = []
    if e > 70: mood_bits.append("high energy")
    elif e < 35: mood_bits.append("low energy")

    if d > 70: mood_bits.append("very danceable")
    elif d < 35: mood_bits.append("chill / not very danceable")

    if v > 65: mood_bits.append("generally happy")
    elif v < 35: mood_bits.append("on the sadder side")

    return " • ".join(mood_bits) or "balanced vibe"

In [7]:
def show_now_playing_and_queue(recs: pd.DataFrame):
    if recs.empty:
        print("No recommendations returned 🙁")
        return

    # row 0 = Now Playing
    now = recs.iloc[0]
    print("🎧 NOW PLAYING")
    print(f"  {now['track_name']} — {now['artists']}")
    print(f"  Genre: {now.get('track_genre', 'N/A')}")
    if "vibe_similarity" in now:
        print(f"   Vibe match: {now['vibe_similarity']:.3f} | Score: {now['vibe_score']:.3f}")
    if "track_id" in now and isinstance(now["track_id"], str):
        url = f"https://open.spotify.com/track/{now['track_id']}"
        display(HTML(f'<p><a href="{url}" target="_blank">▶ Open in Spotify</a></p>'))

    # rows 1–9 = Up Next
    if len(recs) > 1:
        up_next = recs.iloc[1:10][["track_name", "artists", "track_genre", "vibe_score"]]
        print("⬇️ UP NEXT")
        display(up_next.reset_index(drop=True))
    else:
        print("No additional tracks in queue.")

print("Vibe summary:", describe_vibe(sliders))
show_now_playing_and_queue(recs_mode1)

Vibe summary: high energy • very danceable • generally happy
🎧 NOW PLAYING
  Un Coco — Bad Bunny
  Genre: reggaeton
   Vibe match: 0.957 | Score: 0.940


⬇️ UP NEXT


,track_name,artists,track_genre,vibe_score
0,Sobrio,Maluma,reggaeton,0.934318
1,Feel Good Inc.,Gorillaz,alternative,0.929595
2,Se Preparó,Ozuna,reggaeton,0.919002
3,I Ain't Worried,OneRepublic,pop,0.907499
4,Right Now (Na Na Na),Akon,dance,0.899292
5,Satisfya,Imran Khan,hip-hop,0.898673
6,No Me Basta,India Martinez;Dvicio,spanish,0.896389
7,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.888171
8,Black Swan,BTS,k-pop,0.881009


## Mode 2A

In [ ]:
from mode2a_seed_from_song_full import Mode2ASeedFromSong

mode2a = Mode2ASeedFromSong(engine)
mode2a.df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [9]:
query = "feel good inc"  # or "bad bunny", "taylor swift", etc.

candidates = mode2a.search_tracks(query, max_results=10)
display(candidates[["track_name", "artists", "track_genre", "track_id"]])

,track_name,artists,track_genre,track_id
3255,Feel Good Inc.,Gorillaz,alternative,0d28khcov6AiegSCpG5TuT


### Use `df_index` as seed

In [10]:
seed_row = candidates.iloc[0]
seed_track_id = seed_row["track_id"]

print("Using this track_id as seed:", seed_track_id)
print(seed_row[["track_name", "artists", "track_genre"]])

seed_url = f"https://open.spotify.com/track/{seed_track_id}"
print("🔗 Open SEED in Spotify:", seed_url)

now_playing2, up_next2, seed_idx2 = mode2a.recommend_from_seed(
    track_id=seed_track_id,
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

mode2a.print_now_playing_and_queue(
    now_playing=now_playing2,
    up_next=up_next2,
    seed_idx=seed_idx2,
    show_explanation=True,
    show_spotify_embed=True,
)


Using this track_id as seed: 0d28khcov6AiegSCpG5TuT
track_name     Feel Good Inc.
artists              Gorillaz
track_genre       alternative
Name: 3255, dtype: object
🔗 Open SEED in Spotify: https://open.spotify.com/track/0d28khcov6AiegSCpG5TuT
🎧 Now playing: Ferrari — James Hype;Miggy Dela Rosa
▶ Open in Spotify: https://open.spotify.com/track/4zN21mbAuaD0WqtmaTZZeP



Recommended **Ferrari** by James Hype;Miggy Dela Rosa because its vibe closely matches *Feel Good Inc.* on: instrumental vibe 🎼, acoustic feel 🎸, energy ⚡.

🎵 Up Next:


,track_name,artists,track_genre,vibe_score,vibe_similarity
91002,I Ain't Worried,OneRepublic,rock,0.920598,0.910747
89912,Un Coco,Bad Bunny,reggaeton,0.918057,0.930071
20014,One Kiss (with Dua Lipa),Calvin Harris;Dua Lipa,dance,0.916758,0.923448
39408,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.916688,0.935860
91107,Stressed Out,Twenty One Pilots,rock,0.904524,0.918155
85060,Should I Stay or Should I Go - Remastered,The Clash,punk-rock,0.901865,0.927331
65473,Savage,aespa,k-pop,0.896501,0.935627
20960,Personal,HRVY,dance,0.896305,0.945381
1500,La Vuelta Al Mundo,Calle 13,afrobeat,0.894197,0.942746


### Use `track_id` as seed

In [11]:
seed_row = candidates.iloc[0]
seed_track_id = seed_row["track_id"]

print("Using this track_id as seed:", seed_track_id)
print(seed_row[["track_name", "artists", "track_genre"]])

now_playing2, up_next2, seed_idx2 = mode2a.recommend_from_seed(
    track_id=seed_track_id,
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

mode2a.print_now_playing_and_queue(
    now_playing=now_playing2,
    up_next=up_next2,
    seed_idx=seed_idx2,
    show_explanation=True,
)


Using this track_id as seed: 0d28khcov6AiegSCpG5TuT
track_name     Feel Good Inc.
artists              Gorillaz
track_genre       alternative
Name: 3255, dtype: object
🎧 Now playing: Ferrari — James Hype;Miggy Dela Rosa
▶ Open in Spotify: https://open.spotify.com/track/4zN21mbAuaD0WqtmaTZZeP



Recommended **Ferrari** by James Hype;Miggy Dela Rosa because its vibe closely matches *Feel Good Inc.* on: instrumental vibe 🎼, acoustic feel 🎸, energy ⚡.

🎵 Up Next:


,track_name,artists,track_genre,vibe_score,vibe_similarity
91002,I Ain't Worried,OneRepublic,rock,0.920598,0.910747
89912,Un Coco,Bad Bunny,reggaeton,0.918057,0.930071
20014,One Kiss (with Dua Lipa),Calvin Harris;Dua Lipa,dance,0.916758,0.923448
39408,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.916688,0.935860
91107,Stressed Out,Twenty One Pilots,rock,0.904524,0.918155
85060,Should I Stay or Should I Go - Remastered,The Clash,punk-rock,0.901865,0.927331
65473,Savage,aespa,k-pop,0.896501,0.935627
20960,Personal,HRVY,dance,0.896305,0.945381
1500,La Vuelta Al Mundo,Calle 13,afrobeat,0.894197,0.942746


## Mode 2B

In [ ]:
from mode2b_vibe_roulette_full import Mode2BVibeRoulette

In [13]:
engine = SliderVibeRecommender(df_tracks=df_tracks)

# 3. Create Vibe Roulette
roulette = Mode2BVibeRoulette(engine)

# 4. Spin once
now_playing_b, up_next_b, meta_b = roulette.spin(
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

roulette.print_spin_result(now_playing_b, up_next_b, meta_b)

🎲 Vibe Roulette • Thursday • 01:52 PM
Context: Weekday afternoon
Picked persona: **Flow State Focus** — steady, low-distraction beats for deep work
Vibe: mostly instrumental

🎧 Now playing: Судно (Борис Рижий) — Molchat Doma
🔗 Open in Spotify: https://open.spotify.com/track/1SHB1hp6267UK9bJQUxYvO

🎵 Up Next:


,track_name,artists,track_genre,vibe_score,vibe_similarity
56155,YKWIM?,Yot Club,indie-pop,0.910519,0.928149
34209,"Welcome Home, Son",Radical Face,folk,0.903872,0.952340
38462,Cosmic Sass,Good Morning,garage,0.893974,0.979967
57450,Chamber Of Reflection,Mac DeMarco,indie,0.893782,0.912227
79130,Hommage d'amitié,Gunnel Boek,piano,0.881531,0.934413
8916,Ain't No Love In The Heart Of The City - Singl...,"Bobby ""Blue"" Bland",blues,0.878901,0.951126
84214,On the Run,Pink Floyd,psych-rock,0.870754,0.935942
86300,Тоска,Molchat Doma,punk,0.864849,0.916061
36220,Rise Up,Mellowdy,french,0.862947,0.928684


In [22]:
engine = SliderVibeRecommender(df_tracks=df_tracks)
roulette = Mode2BVibeRoulette(engine)

now_playing_b, up_next_b, meta_b = roulette.spin(
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

roulette.print_spin_result(now_playing_b, up_next_b, meta_b)

🎲 Vibe Roulette • Thursday • 02:22 PM
Context: Weekday afternoon

📚💻  Persona: **Flow State Focus**
   steady, low-distraction beats for deep work
   deep work · steady · instrumental

It’s Thursday • 02:22 PM. The algorithm whispers *Flow State Focus*. Starting with **The Night We Met** by Lord Huron to set the tone.
Tagline: steady, low-distraction beats for deep work.
Vibe: balanced vibe.

🎧 Now playing: The Night We Met — Lord Huron
🔗 Open in Spotify: https://open.spotify.com/track/3hRV0jL3vUpRrcy398teAU

▶️ Embedded player:



ℹ️ Why this track?
This track lines up strongly on instrumental vibe 🎼, speechiness 🗣️.

🎵 Up Next:


,track_name,artists,track_genre,vibe_score,vibe_similarity
56155,YKWIM?,Yot Club,indie-pop,0.928631,0.950789
102151,Mystery of Love,Sufjan Stevens,songwriter,0.914670,0.965837
2463,"Call It Fate, Call It Karma",The Strokes,alt-rock,0.906958,0.953698
101000,Herinneringen,Sohn Aelia,sleep,0.888909,0.926136
79324,Horizon,Tamara Eden,piano,0.879669,0.947086
103006,All of Me,John Legend,soul,0.876030,0.882537
80344,Ek Ajnabee Haseena Se,Kishore Kumar;R. D. Burman,pop-film,0.864975,0.926219
2308,The 1975,The 1975,alt-rock,0.861150,0.898937
56974,Krystal,Matt Maltese,indie-pop,0.856214,0.900268
4011,Heavenly,Cigarettes After Sex,ambient,0.854330,0.890412
